In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 15.8 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [ ]:
train_images=train_images/255
test_images=test_images/255

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [39]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [33]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [34]:
tuner_search=RandomSearch(build_model,
                          objective='accuracy',
                          max_trials=3,directory='output',project_name="Mnist Fashion")

In [35]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 3 Complete [00h 00m 27s]
accuracy: 0.9230926036834717

Best accuracy So Far: 0.9230926036834717
Total elapsed time: 00h 01m 52s


In [36]:
model=tuner_search.get_best_models(num_models=1)[0]

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 80)        2080      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        23072     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 112)               1734768   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 1,761,050
Trainable params: 1,761,050
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 4ms/step - loss: 0.1652 - accuracy: 0.9375 - val_loss: 0.2541 - val_accuracy: 0.9105
Epoch 5/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.1283 - accuracy: 0.9514 - val_loss: 0.2707 - val_accuracy: 0.9063
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0987 - accuracy: 0.9643 - val_loss: 0.3265 - val_accuracy: 0.9102
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0761 - accuracy: 0.9720 - val_loss: 0.3332 - val_accuracy: 0.9135
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0590 - accuracy: 0.9781 - val_loss: 0.3806 - val_accuracy: 0.9098
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0459 - accuracy: 0.9837 - val_loss: 0.4467 - val_accuracy: 0.9030
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0393 - accuracy: 0.9851 - val_loss: 0.4657 - val_accuracy